<a href="https://colab.research.google.com/github/stoffprof/X615/blob/main/Wordle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving a word puzzle

In 2018 the *New York Times* introduced a new word game called Spelling Bee. The game board has six letters plus a center letter, and the objective is to construct as many words as possible containing at least the center letter.

As an example, consider this game board:
<p><img src="https://raw.githubusercontent.com/stoffprof/X615/refs/heads/main/images/spelling_bee.png" alt="Alt text" width="200">


Some possible words include *coat*, *bobcat*, and *jackboot*. As is clear from these examples, letters may be reused. Words must be at least four letters. Four-letter words get one point, and longer words earn one point per letter, with a bonus 7 points for "pangrams" that use all letters.

Let's apply what we've learned about data structures to come up with an algorithm that can solve this.



## Using a word corpus

To begin, we need a list of valid words against which we can check a candidate solution. For example, *coat* is a word, but *attcao* isn't; the only way to know this is by checking a list of words.

If we had the contents of, say, the Merriem-Webster dictionary on our computer to look up possible words, we could use that. It is possible to purchase access to dictionaries like that, but there is a free alternative. A number of lists of words, or *corpora*, have been created. Many of these were developed to study how words are used in different contexts. One such corpus is the [Google ngram data](http://storage.googleapis.com/books/ngrams/books/datasetsv2.html), which provides counts of how frequently different unigrams, bigrams, or higher-order *n*-grams appear on the internet. (In computational linguistics, a *unigram* is a single sequence from a body of text, like "word", "3,705" or "ID_054". An example of a bigram is "White House".)

Using unigrams from these data solves one problem--they're free and easy to get--but creates another, which is that not all unigrams are legitimate English words; since they're just pieces of text that appear on web sites, they also include misspellings of words or other short strings.

In [ ]:
import requests

url = ('https://github.com/norvig/pytudes/raw/refs/heads/main/data/text/count_1w.txt')
inf = requests.get(url).text

In [3]:
print('\n'.join(inf.splitlines()[:5]))

the	23135851162
of	13151942776
and	12997637966
to	12136980858
a	9081174698


The file has one record on each line, with the unigram follow by a tab character and the number of times the unigram appears in the corpus. The first record indicates that the word *the* appeared more than 23 billion times on the web pages.

To work with these data, it will be helpful to create a list of the records.

In [4]:
from collections import Counter
unigrams = Counter()

for rec in inf.rstrip().split('\n'):
    w,c = rec.split()
    unigrams[w] = int(c)

unigrams.most_common(10)

[('the', 23135851162),
 ('of', 13151942776),
 ('and', 12997637966),
 ('to', 12136980858),
 ('a', 9081174698),
 ('in', 8469404971),
 ('for', 5933321709),
 ('is', 4705743816),
 ('on', 3750423199),
 ('that', 3400031103)]

The original [n-gram corpus](https://catalog.ldc.upenn.edu/LDC2006T13) contains over 13 million unigrams, but these data are a subset that was created to be more manageable, mostly by excluding unigrams that are quite unlikely to be actual words. It includes only the first one-third of a million ngrams, sorted in descending order of frequency.

In [5]:
len(unigrams)

333333

The `most_common` method prints all elements in decreasing order of their associated count value, so looking at last elements will show us which are the least common.

In [6]:
unigrams.most_common()[-10:]

[('googllr', 12711),
 ('googlal', 12711),
 ('googgoo', 12711),
 ('googgol', 12711),
 ('goofel', 12711),
 ('gooek', 12711),
 ('gooddg', 12711),
 ('gooblle', 12711),
 ('gollgo', 12711),
 ('golgw', 12711)]

Clearly the corpus contains a lot of unigrams that aren't real words. Since it was created automatically from web page content and isn't systematically curated, there are a lot of things on the list that aren't actually words.

We can try to remove entries that are either not real words, or are extremely obscure, by keeping only those that appear more than, say, 100,000 times. This cutoff is obviously arbitrary. Some non-words likely appear more than this, while some real words may not meet this cutoff. For example, here's a long word that is really a word, but isn't used that frequently online.

In [7]:
unigrams['antidisestablishmentarianism']

16489

In [8]:
wordlist = [word for word,count in unigrams.items() if count>100000]

Because we're unpacking the (*word*, *count*) pairs we need to be careful to remove the last entry in the list to avoid getting a "not enough values to unpack" error, as it only has one entry. We're left with a list of just under 100,000 words.

In [9]:
len(wordlist)

99487

There is a little more pruning of our list we can do. First, some unigrams have no vowels, but any real word needs them. Here's a unigram that appears quite frequently but isn't a word.

In [10]:
unigrams['bbb']

2710841

To filter these out, we'll make a small function that checks if a word contains any vowels.

In [11]:
def has_vowel(word):
    return any([v in word for v in list('aeiou')])

In [12]:
wordlist = list(filter(has_vowel, wordlist))
len(wordlist)

95038

That removed about 4,000 unigrams. Another restriction we can impose is that words be at least four letters long, since this is one of the rules of the puzzle. This removes about 5,000 more words from our list, leaving us with just over 90,000 words.

In [13]:
wordlist = [word for word in wordlist if len(word)>3]
len(wordlist)

90223

## A word-finding algorithm

How can we go about finding word candidates that appear on the list? A brute force approach would be to generate possible words and check if they are in the dictionary. While this could work, it would be incredibly inefficient; if we only check words up to ten letters long, there are still about 330 million possible words to check:

In [18]:
[7**k for k in range(1,11)]

[7, 49, 343, 2401, 16807, 117649, 823543, 5764801, 40353607, 282475249]

In [19]:
'{:,d}'.format(sum([7**k for k in range(1,11)]))

'329,554,456'

How long would it take to check whether 330 million possible words are in the dictionary? We can estimate this by checking how long it takes to look up one word.

In [15]:
%timeit 'belicose' in wordlist

1.34 ms ± 238 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The `%timeit` macro repeatedly executes a piece of code and times how long each execution takes. In this case, it checked whether *belicose* is in the list 1000 times, and found that on average it took about one millisecond. That's pretty fast, but if we had to to it 330 million times, it would take almost four days:

In [16]:
# convert milliseconds to days
329554457 / 1000 / 60 / 60 / 24

3.814287696759259

Clearly, we need a better approach. One idea is to preprocess our word list to construct a list of words that can be constructed from all possible letter groupings. For example, we might have:

`words[('a','b','c','k')] = ['back', 'aback']`

`words[('b','e','l','t')] = ['belt', 'beetle']`

We could then generate all possible unique combinations of our 7 letters, and then simply look in our dictionary for words that match the necessary pattern. We can easily construct such a dictionary from our word list by using a `defaultdict`.

In [ ]:
from collections import defaultdict
words = defaultdict(list)
for word in wordlist:
    words[tuple(sorted(set(word)))].append(word)

To see how this works, consider each of the pieces of the code. We iterate over our list of words, assigning each one to `word` one-at-a-time. We then call `set()` on `word`, which will return a set of the unique letters in the word. (Recall that `set` takes an iterable--in this case a string--and creates a set with the unique elements.) We then sort the elements of the set, which returns a list, and finally convert this list to a tuple so it can be used as the key in the `words` dictionary. The value corresponding to this key is a list, and we append `word` to this list.

Having done this, we have a dictionary of 47,500 possible letter combinations.

In [ ]:
len(words)

Let's look at a few examples to see what the dictionary looks like.

In [ ]:
words[('a', 'b', 'c', 'k')]

In [ ]:
words[('a', 'b', 'c', 'k', 'l')]

We obviously didn't get rid of all non-words with our filters. *balck* is likely a misspelling of *black* that appears sufficiently frequently on the internet that it has made it into our list. Similarly, *kabc* obviously isn't a word. It is in our corpus, though, perhaps because KABC is the ABC television affiliate in Los Angeles.

Since the keys are sorted letters, trying to look up a key where the letters are not sorted won't return any words.

In [ ]:
words[('b', 'c', 'a')]

This was intentional, though. We need the keys to be standardized so if we want to look up the letter combination `('a', 't', 'c')` we know which order to put them in. This is why we sorted the letters in the tuple before using them as a key when we created the `words` dictionary.

We're almost ready to solve the puzzle. We'll store the center letter in a variable `lc` and the outer letters in a list `lo`. Using the letters from the game board printed above, we have:

In [ ]:
lc = 'c'
lo = list('btjoka')

We want a list of all possible combinations of these letters that always include the center letter. For example, we'll want to try words containing `[a, b, c]`, `[a, c, t]`, `[a, b, c, t]`, and so on.

If we wanted to improve the efficiency of our code, we could also require that each word has at least one vowel. There's no point looking for words containing only the letters `[b, c, t]` because there aren't any. The current problem is small enough, however, that the benefit of doing that is likely to be quite small, so there's no need to incorporate this into our code.

The built-in `itertools` module has various functions for getting permutations and combinations from an iterable. We'll use the `combinations(iter, len)` function, which returns all combinations from `iter` of length `len`. For our purposes, we want all combinations of the center letter combined with one or more of the other letters, so we'll choose combinations of `lo` with a length that varies from 1 to 6.

In [ ]:
from itertools import combinations
combs = [tuple(sorted((lc,)+comb))
         for x in range(1,6)
         for comb in combinations(lo,x)]

`combs` is now a list of possible letter combinations that can be constructed from our 7 letters. Here are the first and last several elements:

In [ ]:
combs[:3]

In [ ]:
combs[-3:]

We're now ready to look in our word list for each of the candidate letter combinations. We'll iterate over the possible combinations and get the list of words that match the given letter pattern.

In [ ]:
rslt = []
for comb in combs:
    for word in words[comb]:
        rslt.append(word)
print(sorted(rslt, key=len, reverse=True))

This works well in finding solutions like *cookbook* or *cockatoo*, but also includes things that aren't words, like *otcbb* and *acto*. Unfortunately there's no good way to fix this without improving our corpus. But we have pretty easily come up with a good list of candidate solutions.

Let's put everything together into a function that can solve an arbitrary puzzle.

In [ ]:
def solve_spelling_bee(lc, lo, words, minlen=4):
    [lc, lo] = map(str.lower, [lc, lo])
    lo = list(lo)
    combs = [tuple(sorted((lc,)+comb))
             for x in range(1,6)
             for comb in combinations(lo,x)]
    rslt = []
    for comb in combs:
        for word in words[comb]:
            if len(word)>minlen: rslt.append(word)
    return sorted(rslt, key=len, reverse=True)

In [ ]:
print(solve_spelling_bee('L', 'ACITVY', words, 5))

In [ ]:
print(solve_spelling_bee('W', 'GLEHTI', words))

As a final step, we can check each of these candidate solutions in an online dictionary. Querying a web server over the internet is much, much slower than processing data within our computer, so this approach has to be done carefully and when there aren't alternative methods available.

Here I'm using *The Free Dictionary* to look up words. Before writing this code I use my web browser to try a few words in the dictionary and see how the web site presents results when a word is--or is not--found.

In [ ]:
def is_word(word):
    import requests
    url = 'https://www.thefreedictionary.com/'
    resp = requests.get(url + word)
    if resp.ok:
        if resp.url == url + word:
            return False if ('Word not found in the Dictionary' in
                            resp.text) else True
        else:
            return False

A quick test of the function confirms that it works as expected.

In [ ]:
is_word('bucolic')

In [ ]:
is_word('otcbb')

Note that after checking that the response is valid (that is, that `resp.ok` is `True`), I check that the URL of the response is the same as in the request. I included this because I noticed that the web site sometimes presents results from an encyclopedia when it doesn't find the word in the dictionary, and I want to exclude those.

We can now iterate over the list of solutions from the corpus, check each in the online dictionary, and keep only those that appear there. In this example, we reduce the number of solutions from 34 to 19. Since we're pausing for one second between each iteration, this code will take a little longer to execute.

In [ ]:
from time import sleep
solutions = []
for cand in solve_spelling_bee('W', 'GLEHTI', words):
    if is_word(cand):
        solutions.append(cand)
        sleep(1)

print(solutions)

You may be tempted to write more succinct code here by using a list comprehension, like this:

In [ ]:
[cand for cand in solve_spelling_bee('W', 'GLEHTI', words)
 if is_word(cand)]

Doing so, however, would prevent us from calling the sleep function to slow down the repeated web server requests. It is actually possible to build the throttling into the solve_spelling_bee function, but doing so requires some additional features that we will introduce later.

Finally, we'll revisit our function, adding the online dictionary lookup along with some additional bells and whistles.

In [ ]:
def solve_spelling_bee(lc, lo, words, minlen=4, wait=1):
    """Solves a Spelling Bee game

    Parameters
    ----------
    lc : str
        single letter at center of board
    lo : str
        5-character string of other letters
    words : dict
        dictionary with letter combinations as keys and
        list of words as values
    minlen : int
        minimum length of returned words
    wait : float
        number of seconds to pause between queries to
        online dictionary
    """
    from itertools import combinations

    # Require inputs to be correct types
    if not (isinstance(lc, str) and len(lc)==1):
        print('Must provide a single center letter <lc>')
        return None
    if not (isinstance(lo, str) and len(lo)==6):
        print('Must provide a list of 6 letters <lo>')
        return None
    if not isinstance(words, dict):
        print('Must provide a dictionary of words')
        return None

    [lc, lo] = map(str.lower, [lc, lo])
    lo = list(lo)
    combs = [tuple(sorted((lc,)+comb))
             for x in range(1,6)
             for comb in combinations(lo,x)]
    cands = []
    for comb in combs:
        for word in words[comb]:
            if (len(word)>minlen): cands.append(word)
    print(f'Found {len(cands)} candidate solutions.')

    print('Checking online dictionary...', end=' ')

    rslt = []
    for cand in cands:
        if is_word(cand):
            rslt.append(cand)
        sleep(wait)
    print(f'Found {len(rslt)} solutions.')
    return sorted(rslt, key=len, reverse=True)

In [ ]:
solve_spelling_bee('W', 'GLEHTI', words, wait=0.25)